In [14]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import os
import datetime as dt


from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
print(BASE_DIR)
import scripts.functions as fn

from scripts.indicators import Fibonacci
from scripts.indicators import resample, zigzag, fibonacci_retroceso, fibonacci_extension

from binance.client import Client

C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app
C:\Users\lbisa\Mi unidad (leonardo.bisaro@gmail.com)\Cripto\python\trd_app


In [15]:
#Funcion para descarga de velas
def get_klines(symbol,interval,start,end):
    print(symbol, interval, start, end )
    df = client.get_historical_klines(symbol=symbol, interval=interval, start_str=start, end_str=end)
    df = pd.DataFrame(df)
    df = df.iloc[:, :6]
    df.columns = ["datetime", "open", "high", "low", "close", "volume"]
    df['open'] = df['open'].astype('float')
    df['high'] = df['high'].astype('float')
    df['low'] = df['low'].astype('float')
    df['close'] = df['close'].astype('float')
    df['volume'] = df['volume'].astype('float')
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms') - dt.timedelta(hours=3)
    #df=df.set_index('datetime')
    return df

In [16]:
environment = 'klines'
#environment = 'binance'

if environment == 'klines':

	klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
	with open(klines_file, 'rb') as file:
		df = pickle.load(file)
		#df.set_index('datetime',inplace=True)

	df = resample(df,(60*4))
else:
	#Conexion con Binance sin API-KEY
	client = Client()

	start = '2024-6-01'
	end = '2024-11-24'
	symbol = 'BTCUSDT'
	interval = '1d'
	real_start = (dt.datetime.strptime(start, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	real_end = (dt.datetime.strptime(end, "%Y-%m-%d") ).strftime("%Y-%m-%d")
	utc_start = real_start+' 00:00:00'
	utc_end = real_end+' 23:59:59'
	df = get_klines(symbol=symbol,interval=interval, start=utc_start,end=utc_end)


#df = fn.ohlc_mirror_v(df)

#Filtrar rango de fechas
#ini = pd.to_datetime('2018-06-26')
#fin = pd.to_datetime('2028-05-01')
#df = df[(df['datetime']>=ini)&(df['datetime']<=fin)]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)

#df.drop(columns=['volume'],inplace=True)

#Quitar parte del dataframe
#df = df[500:1500]
#df.reset_index(inplace=True)
#df.drop(columns=['index'],inplace=True)


In [ ]:
# Aplicar la función signal
df = zigzag(df, resample_periods=4)

#Detectando pivots para analisis de fibonacci
pivots = df[df['ZigZag']>0].copy()
pivots['pivots'] = pivots['ZigZag'].copy()
pivots['fb_0'] = pivots['ZigZag'].copy()
pivots['fb_1'] = pivots['ZigZag'].shift(1)
pivots['fb_2'] = pivots['ZigZag'].shift(2)

#Cambios de tendencia
pivots['trend'] = np.where((pivots['fb_0']>pivots['fb_2']) & (pivots['fb_1']>pivots['fb_0']), 2,0)
pivots['trend'] = np.where((pivots['fb_0']>pivots['fb_2']) & (pivots['fb_1']<pivots['fb_0']), 1,pivots['trend'])
pivots['trend'] = np.where((pivots['fb_0']<pivots['fb_2']) & (pivots['fb_1']<pivots['fb_0']),-2,pivots['trend'])
pivots['trend'] = np.where((pivots['fb_0']<pivots['fb_2']) & (pivots['fb_1']>pivots['fb_0']),-1,pivots['trend'])
first_pivot_index = pivots.index[0]
pivots.at[first_pivot_index,'trend'] = 0 #El primer pivot no se puede definir por lo tanto queda neutro

#Completando el datafame principal con los datos de pivots
df_cols = df.columns.copy()
for col in pivots.columns:
	if not col in df_cols:
		df[col]=None
for index, row in pivots.iterrows():
	for col in pivots.columns:
		if not col in df_cols:
			df.at[index,col] = row[col]

df['trend'].ffill(inplace=True)
df['fb_0'].ffill(inplace=True)
df['fb_1'].ffill(inplace=True)
df['fb_2'].ffill(inplace=True)

df['long_fbe_0'] = np.where(df['trend']==2, df['fb_0'] , None)
df['long_fbe_1'] = np.where(df['trend']==2, df['fb_1'] , None)
df['long_fbe_2'] = np.where(df['trend']==2, df['fb_2'] , None)




##Retroceso de Fibonacci
#fbr_levels = [0.382, 0.618, 0.786]
#for level in fbr_levels:
#	df[f'long_fbr_{level}'] = df.apply(
#    	lambda row: fibonacci_retroceso(row["long_fbr_1"], row["long_fbr_0"], level)
#    	if pd.notnull(row["long_fbr_0"]) and pd.notnull(row["long_fbr_1"])
#    	else None,
#    	axis=1
#	)


#Extension de Fibonacci
fbe_levels = [0.236, 0.382, 0.5, 0.618, 0.786, 1.272, 1.618, 2.0, 2.618]
for level in fbe_levels:
	df[f'long_fbe_{level}'] = df.apply(
    	lambda row: fibonacci_extension(row["fb_2"], row["fb_1"], row["fb_0"], level)
    	if (pd.notnull(row["fb_2"]) and row['trend'] == 2)
    	else None,
    	axis=1
	)

In [18]:
df_to_show = df #[0:20]

indicators = [
     {'col': 'ZigZag','name': 'ZigZag','color': 'gray','row': 1, 'mode':'lines',},
     {'col': 'long_fbe_0','name': 'long_fbe_0','color': 'red','row': 1, 'mode':'markers',},
     {'col': 'long_fbe_1','name': 'long_fbe_1','color': 'white','row': 1, 'mode':'markers',},
     #{'col': 'fb_2','name': 'fb_2','color': 'red','row': 2, 'mode':'markers',},
     #{'col': 'stop_loss','name': 'stop_loss','color': 'gray','row': 1, 'mode':'markers',},
     #{'col': 'take_profit','name': 'take_profit','color': 'gray','row': 1, 'mode':'markers',},
        ]
#for level in fbr_levels:
#    indicators.append({'col': f'long_fbr_{level}','name': f'long_fbr_{level}','color': 'gray','row': 1, 'mode':'markers',})
for level in fbe_levels:
    indicators.append({'col': f'long_fbe_{level}','name': f'long_fbe_{level}','color': 'gray','row': 1, 'mode':'markers',})
    
indicators_out = [
     {'col': 'trend','name': 'trend','color': 'yellow','row': 1, 'mode':'markers',},
     #{'col': 'maxs_up','name': 'maxs_up','color': 'green','row': 2, 'mode':'lines',},
        ]

#https://plotly.com/python/reference/scatter/#scatter-marker-symbol
events = [
       #{'df':df_to_show,'col':'buy','name': 'BUY','color': 'green','symbol': 'triangle-up' }, 
       #{'df':df_to_show,'col':'sell','name': 'SELL','color': 'red','symbol': 'triangle-down' }, 
     ]


fig = fn.ohlc_chart(df_to_show,indicators=indicators,indicators_out=indicators_out,events=events,) 
fig.show()
